In [5]:
# Traffic Accident Analysis and Prediction


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [56]:
import os
print(os.listdir('.'))


['.anaconda', '.bash_history', '.cache', '.conda', '.condarc', '.config', '.continuum', '.emulator_console_auth_token', '.git', '.gitconfig', '.gitignore', '.gnutls', '.gradle', '.idea', '.ipynb_checkpoints', '.ipython', '.jdks', '.jupyter', '.keras', '.m2', '.matplotlib', '.ms-ad', '.nbi', '.netbeans-derby', '.node_repl_history', '.opera', '.redhat', '.thumbnails', '.virtual_documents', '.vscode', '.vscode-R', '345', '3D Objects', '890', 'ActivityLayouts', 'Amazon_Clone_Project', 'AndriodCode', 'AndroidCode', 'AndroidStudioProjects', 'app', 'AppData', 'Application Data', 'Casualties.csv', 'Contacts', 'Cookies', 'custom.css', 'data', 'Demo', 'Documents', 'Downloads', 'Favorites', 'git', 'htmll.html', 'index.html', 'java_error_in_studio64.hprof', 'kislay.wav', 'Links', 'Local Settings', 'local.properties', 'MicrosoftEdgeBackups', 'Music', 'My Documents', 'NetHood', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{dcf7e46f-52d4-11ec-8118-8b57a4f2d403}.TM.blf', 'NTUSER.DAT{

In [2]:
df = pd.read_csv('data/Casualties.csv')
df.head(20)


NameError: name 'pd' is not defined

In [ ]:
df.describe()
df.info()


In [ ]:
df.isnull().sum()


In [ ]:
# Distribution of a feature
sns.histplot(df['Accident_Index'])
plt.show()



In [ ]:
# Boxplot of features
sns.boxplot(x='Accident_Index', y='Casualty_Class', data=df)
plt.show()


In [ ]:
# Identify numeric columns
numeric_columns = df.select_dtypes(include=['number']).columns

# Fill missing values in numeric columns with their mean
df=df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())

In [ ]:
df

In [ ]:
# Example: Creating a new feature
df['new_feature'] = df['Casualty_Class'] / df['Age_of_Casualty']
df['new_feature']




In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib


In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop('Age_of_Casualty', axis=1)
y = df['Casualty_Reference']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))


In [ ]:
import joblib
from sklearn.ensemble import RandomForestClassifier
joblib.dump(model, 'trained_model.pkl')


In [ ]:
pip install fastapi uvicorn


In [ ]:
pip install python-multipart

In [ ]:
from fastapi import FastAPI, File, UploadFile
import joblib


app = FastAPI()
model = joblib.load('trained_model.pkl')

@app.post("/predict/")
async def predict(file: UploadFile):
    # Process file and make predictions
    return {"prediction": "result"}


In [ ]:
!uvicorn main:app --reload



In [ ]:
# Compute the correlation matrix
correlation_matrix = df.corr()

# Generate a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()


In [ ]:
# Pair plot for selected features
from sklearn.model_selection import GridSearchCV
sns.pairplot(df[['Age_of_Casualty', 'Age_Band_of_Casualty', 'Casualty_Severity']])
plt.show()


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters and best score
print('Best parameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict on test set
y_pred = grid_search.best_estimator_.predict(X_test)

# Classification report
print(classification_report(y_test, y_pred))

# Confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
pip install pandas


In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import HTMLResponse
import joblib
import pandas as pd

nest_asyncio.apply()

app = FastAPI()
model = joblib.load('trained_model.pkl')

# Replace with the actual columns used for training
required_columns = ['Accident_Index', 'feature2', 'feature3']  # Example column names, update as needed

@app.get("/", response_class=HTMLResponse)
async def read_root():
    html_code = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Traffic Accident Prediction</title>
    </head>
    <body>
        <h1>Traffic Accident Prediction</h1>
        <form action="/predict/" method="post" enctype="multipart/form-data">
            <input type="file" name="file" accept=".csv">
            <button type="submit">Upload and Predict</button>
        </form>
    </body>
    </html>
    """
    return HTMLResponse(content=html_code)

@app.post("/predict/")
async def predict(file: UploadFile):
    try:
        df = pd.read_csv(file.file)

        # Print columns of the uploaded file
        print("Uploaded file columns:", df.columns.tolist())

        # Check if all required columns are present and no extra columns are included
        missing_columns = [col for col in required_columns if col not in df.columns]
        extra_columns = [col for col in df.columns if col not in required_columns]

        if missing_columns:
            return {"error": f"Uploaded file is missing required columns: {', '.join(missing_columns)}"}
        if extra_columns:
            return {"error": f"Uploaded file contains unexpected columns: {', '.join(extra_columns)}"}

        df = df[required_columns]  # Ensure the columns are in the correct order
        df.fillna(df.mean(numeric_only=True), inplace=True)

        predictions = model.predict(df)
        return {"predictions": predictions.tolist()}

    except pd.errors.EmptyDataError:
        return {"error": "Uploaded file is empty."}
    except pd.errors.ParserError:
        return {"error": "Error parsing the CSV file."}
    except Exception as e:
        return {"error": str(e)}

# Run FastAPI server
uvicorn.run(app, host="127.0.0.1", port=8001)





In [ ]:
!uvicorn Trafic_Accident_Prediction:app --reload
